In [1]:
from data_tools.api import *
import pymysql
import pandas as pd
import numpy as np

DB_INFO = dict(host='192.168.1.234',
               user='winduser',
               password='1qaz@WSX',
               db='wind')
conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

date = 20191101

In [1]:
from data_tools.api import trade_days

trade_dates_all = trade_days.copy()
def get_prev_n_trade_date(trade_date, n):
    pos = np.searchsorted(trade_dates_all, trade_date)
    assert pos >= n
    return int(trade_dates_all[pos - n])

def get_next_n_trade_date(trade_date, n=1):
    pos = np.searchsorted(trade_dates_all, trade_date, side='right')
    assert pos + n - 1 < len(trade_dates_all)
    return int(trade_dates_all[pos + n - 1])

| 二级分类   | 简称             | 因子构建方式                                                                                                | 财报数据类型 | 是否哑变量 |
|--------|----------------|-------------------------------------------------------------------------------------------------------|--------|-------|
| 关联交易   | risk\_关联进货销货   | （向关联方采购产品金额合计\+向关联方销售产品金额合计）/净资产绝对值                                                                   | \-     | \-    |
| 关联交易   | risk\_关联融资余额   | 向关联方提供资金余额/净资产绝对值                                                                                     | \-     | \-    |
| 关联交易   | risk\_关联并购     | 3年内关联并购总金额/过去3年平均总资产                                                                                  | \-     | \-    |
| 关联交易   | risk\_关联交易次数   | 1年内公司关联交易记录次数                                                                                         | \-     | \-    |
| 分析师预期  | risk\_评级周下调    | 近一周分析师评级下调                                                                                            | \-     | \-    |
| 分析师预期  | risk\_目标价周下调   | 近一周分析师目标价下调                                                                                           | \-     | \-    |
| 分析师预期  | risk\_预期EPS周下调 | 近一周分析师预期每股收益下调（最近预测年度FY1）                                                                             | \-     | \-    |
| 减持     | risk\_大股东减持    | 近3个月大股东减持比例，减持比例=减持股票数/总股本                                                                            | \-     | \-    |
| 减持     | risk\_高管减持     | 近3个月高管减持比例，减持比例=减持股票数/总股本                                                                             | \-     | \-    |
| 重组失败   | risk\_重组失败     | 近6个月是否有重大资产重组失败记录，有为1分，无为0分                                                                           | \-     | 哑变量   |
| 限售股解禁  | risk\_限售股解禁    | 未来1个月内限售股解禁比例，解禁比例=解禁股票数/总股本                                                                          | \-     | \-    |
| 债券负面   | risk\_债券违约     | 近2年内是否有债券违约，有为1分，无为0分（担保违约、担保人代偿、兑付风险警示、未按时兑付本金...） | \-     | 哑变量   |
| 负面公告   | risk\_问询函      | 近一个月是否收到问询函，有为1分，无为0分                                                                                 | \-     | 哑变量   |
| 预快报绩效  | risk\_业绩预亏     | 关于下一期业绩的业绩预告是否显示了公司净利润为负，是为1分，不是为0分                                                                   | \-     | 哑变量   |
| 定期报告绩效 | risk\_审计意见     | 财报审计意见是否为标准无保留意见，不是记为1分，是记为0分                                                                         | \-     | 哑变量   |
| 定期报告绩效 | risk\_政府补助     | 最新一期政府补助比净利润                                                                                          | 最新     | \-    |
| 预快报绩效  | risk\_业绩预告大幅下滑 | 关于下一期业绩的业绩预告是否显示了公司未来及利润将大幅下滑（50%以上），上限与下限的平均，是为1分，不是为0分                                              | \-     | 哑变量   |
| 定期报告绩效 | risk\_业绩实亏     | 最新一期净利润是否为负，是为1分，不是为0分                                                                                | 最新     | 哑变量   |


In [3]:
#公司代码和股票代码对照表
print(pd.datetime.now())
sql="SELECT S_INFO_WINDCODE, S_INFO_COMPCODE from WINDCUSTOMCODE where S_INFO_SECURITIESTYPES='A'"
print('Reading sql WindCustomCode')
df_code_map = pd.read_sql_query(sql,conn)
df_code_map.rename({'S_INFO_WINDCODE': 'sid', 'S_INFO_COMPCODE': 's_info_compcode'}, axis=1, inplace=True)
df_code_map.sort_values('sid', inplace=True)
print(pd.datetime.now())

assert len(np.unique(df_code_map.sid)) == len(df_code_map)
code_map = {ele['s_info_compcode']: ele['sid'] for _, ele in df_code_map.iterrows()}

2019-11-29 00:11:23.786651
Reading sql WindCustomCode
2019-11-29 00:11:29.472850


In [4]:
# 关联交易（关联进货销货，关联融资余额，关联并购，关联交易次数）
def get_related_trade_info(date, conn):
    ###输出为一个元组，包含4个sr
    date_p1y = get_prev_n_trade_date(date, 240)
    date_p3y = get_prev_n_trade_date(date, 720)
    def func_ratio(a, b, eps=10000.0):
        return a / (np.abs(b) + eps)
#         return (a - b) / (np.abs(a) + np.abs(b) + eps) 
    
    # 获取财务指标
    sql=(f"""select S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD, TOT_ASSETS, TOT_LIAB from ASHAREBALANCESHEET"""
         + f""" where STATEMENT_TYPE='408001000' and REPORT_PERIOD>='%s' and ANN_DT<='%s'""" % (date_p3y, date))
    print('Reading sql AShareBalanceSheet')
    df_balance = pd.read_sql(sql,conn)
    df_balance.columns = [ele.lower() for ele in df_balance.columns]
    df_balance.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_balance.ann_dt = df_balance.ann_dt.astype('i4')
    df_balance.report_period = df_balance.report_period.astype('i4')
    df_balance = df_balance[df_balance.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))
                            & (~np.isnan(df_balance.tot_assets)) & (~np.isnan(df_balance.tot_liab))]
    df_balance.sort_values(['sid', 'report_period', 'ann_dt'], inplace=True)
    ## 处理数据
    df_balance['net_assets'] = df_balance.tot_assets - df_balance.tot_liab
    df_balance_p1y = df_balance[df_balance.report_period >= date_p1y]
    sr_tot_assets_p3y = df_balance.groupby('sid', as_index=True)['tot_assets'].mean()  # 3年总资产平均值
    sr_net_assets = df_balance_p1y.groupby('sid', as_index=True)['net_assets'].last()  # 近期的净资产
    
    # 关联进货销货
    sql = ("""select S_INFO_WINDCODE, ANN_DT, S_RELATEDTRADE_TRADETYPE, CRNCY_CODE, S_RELATEDTRADE_AMOUNT from ASHARERALATEDTRADE"""
           + f""" where ANN_DT>='%s' and ANN_DT<='%s'""") % (date_p1y, date)
    print('Reading sql AShareRalatedTrade')
    df_related_trade = pd.read_sql(sql,conn)
    df_related_trade.columns = [ele.lower() for ele in df_related_trade.columns]
    df_related_trade.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_related_trade.ann_dt = df_related_trade.ann_dt.astype('i4')
    df_related_trade['s_relatedtrade_amount'] = df_related_trade['s_relatedtrade_amount'].replace([None, np.nan], '0').astype('f8')  # wind给的type是字符串
    df_related_trade = df_related_trade[df_related_trade.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_related_trade.sort_values(['sid', 'ann_dt'], inplace=True)
    #处理数据，通过对照公告，s_relatedtrade_amount单位为元
    type_sell_buy = ['出售', '向关联方采购产品和接受劳务', '向关联方销售产品和提供劳务', 
                     '接受劳务', '接受服务', '提供加工', '提供劳务', '提供服务', '购买', '购买商品', '购销', '购销,提供服务', '购销商品', 
                     '采购', '采购货物', '采购货物,接受劳务', '销售', '销售,提供', '销售,提供劳务', '销售商品', '销售货物']
    df_related_sellbuy = df_related_trade[df_related_trade.s_relatedtrade_tradetype.isin(type_sell_buy)
                                          & df_related_trade.crncy_code.apply(lambda x: x in ['CNY', None])]
    sr_related_sellbuy_amount = df_related_sellbuy.groupby('sid', as_index=True)['s_relatedtrade_amount'].sum()
    sr_related_sellbuy_ratio = func_ratio(sr_related_sellbuy_amount, sr_net_assets)   # 关联进货销货
    
    # 关联融资余额, 也有报告期
    sql = ("""select S_INFO_COMPCODE, ANN_DT, REPORT_PERIOD, ASSOCIATED_NAME, ASSOCIATED_FUNDING_BALANCE, CRNCY_CODE from ASHARERELATEDCLAIMSDEBTS """
           + """ where REPORT_PERIOD>='%s' and ANN_DT<='%s'""" % (date_p1y, date))
    print('Reading sql AShareRelatedclaimsdebts')
    df_related_debts = pd.read_sql(sql,conn)
    df_related_debts.columns = [ele.lower() for ele in df_related_debts.columns]
    df_related_debts.ann_dt = df_related_debts.ann_dt.astype('i4')
    df_related_debts.report_period = df_related_debts.report_period.astype('i4')
    df_related_debts['sid'] = [code_map.get(ele, '0') for ele in df_related_debts.s_info_compcode]
    df_related_debts = df_related_debts[df_related_debts.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_related_debts.sort_values(['sid', 'report_period'], inplace=True)
    ## 处理数据， 通过对照20190630报告期，000005.SZ和	603888.SH，可以推断余额是万元单位
    df_related_debts = df_related_debts[df_related_debts.crncy_code.apply(lambda x: x in ['CNY', None])]
    sr_related_debts_balance = 10000.0 * df_related_debts.groupby(['sid', 'report_period'], as_index=True)['associated_funding_balance'].sum().groupby('sid').last()
    sr_related_debts_ratio = func_ratio(sr_related_debts_balance, sr_net_assets)  # 关联融资余额
    
    # 关联并购
    sql="""select EVENT_ID, S_INFO_WINDCODE from MERGERPARTICIPANT"""  # 获取关联并购的参与方
    print('Reading sql MergerParticipant')
    df_merger_participant = pd.read_sql(sql,conn)
    df_merger_participant.columns = [ele.lower() for ele in df_merger_participant.columns]
    df_merger_participant.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_merger_participant = df_merger_participant[df_merger_participant.sid.apply(lambda x:x is not None and ((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_merger_participant = df_merger_participant.groupby(['event_id', 'sid'], as_index=True).last().reset_index()
    # 获取关联并购
    sql = ("""select EVENT_ID, ANN_DATE, UPDATE_DATE, IS_RELATED_PARTY_TRANSAC, PROGRESS_CODE, TRADE_VALUE, CRNCY_CODE from MERGEREVENT"""
           + """ where IS_RELATED_PARTY_TRANSAC=1 and UPDATE_DATE>='%s' and UPDATE_DATE<='%s'""" % (date_p3y, date))
    print('Reading sql MergerEvent')
    df_related_merger = pd.read_sql(sql,conn)
    df_related_merger.columns = [ele.lower() for ele in df_related_merger.columns]
#     df_related_merger.ann_date = df_related_merger.ann_date.astype('i4'), not use
    df_related_merger.update_date = df_related_merger.update_date.astype('i4')
    df_related_merger = df_related_merger.merge(df_merger_participant, on='event_id',how='inner')  #合并，取inner
    df_related_merger = df_related_merger[df_related_merger.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_related_merger.sort_values(['sid', 'update_date'], inplace=True)
    # 处理数据
    df_related_merger = df_related_merger[df_related_merger.crncy_code.apply(lambda x: x in ['CNY', None])
                                          & df_related_merger.progress_code.apply(lambda x: x in [324004000, 324004001, 324004002, 324004003])]
    df_related_merger_amount = 10000.0 * df_related_merger.groupby('sid', as_index=True)['trade_value'].sum()
    df_related_merger_ratio = func_ratio(df_related_merger_amount, sr_tot_assets_p3y)  # 关联并购
    
    #  关联交易次数
    df_related_trade_num = df_related_trade.groupby('sid', as_index=True).apply(len)
    
    # 输出结果
    return sr_related_sellbuy_ratio, sr_related_debts_ratio, df_related_merger_ratio, df_related_trade_num

In [5]:
print(pd.datetime.now())
a,b,c,d = get_related_trade_info(date, conn)
print(pd.datetime.now())

2019-11-29 00:11:30.311224
Reading sql AShareBalanceSheet
Reading sql AShareRalatedTrade
Reading sql AShareRelatedclaimsdebts
Reading sql MergerParticipant
Reading sql MergerEvent
2019-11-29 00:11:51.260379


In [6]:
## 分析师预期（评级周下调，目标价周下调，预期eps周下调）
def get_analyst_expect_info(date, conn):
    date_p5d = get_prev_n_trade_date(date, 5)
    date_p1m = get_prev_n_trade_date(date, 20)
    date_p3m = get_prev_n_trade_date(date, 60)  #先获取近3个月的数据
    
    # 获取评级周下调, 分值越低评级越高，周期有三个30天、90天河180天先不管
    sql=("""select S_INFO_WINDCODE, RATING_DT, S_WRATING_CYCLE, S_WRATING_AVG, S_EST_PRICE from ASHARESTOCKRATINGCONSUS"""
         + """ where RATING_DT>='%s' and RATING_DT<='%s'""" % (date_p3m, date))
    print('Reading sql AShareStockRatingConsus')
    df_stock_rating=pd.read_sql(sql,conn)
    df_stock_rating.columns = [ele.lower() for ele in df_stock_rating.columns]
    df_stock_rating.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_stock_rating.rating_dt = df_stock_rating.rating_dt.astype('i4')
    df_stock_rating = df_stock_rating[df_stock_rating.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_stock_rating.sort_values(['sid', 'rating_dt', 's_wrating_cycle'], inplace=True)
    # 评级周下调
    df_stock_rating_early = df_stock_rating[df_stock_rating.rating_dt <= date_p5d]
    df_stock_rating_p5d = df_stock_rating[df_stock_rating.rating_dt > date_p5d]
    df_stock_rating_early2 = df_stock_rating_early[~np.isnan(df_stock_rating_early.s_wrating_avg)]
    df_stock_rating_p5d2 = df_stock_rating_p5d[~np.isnan(df_stock_rating_p5d.s_wrating_avg)]
    sr_rating_early = df_stock_rating_early2.groupby('sid', as_index=True)['s_wrating_avg'].last()
    sr_rating_last = df_stock_rating_p5d2.groupby('sid', as_index=True)['s_wrating_avg'].last()
    sr_rating_delta = sr_rating_last - sr_rating_early  # 大于0表示评级下降
    
    # 目标价周下调
    df_stock_rating_early3 = df_stock_rating_early[~np.isnan(df_stock_rating_early.s_est_price)]
    df_stock_rating_p5d3 = df_stock_rating_p5d[~np.isnan(df_stock_rating_p5d.s_est_price)]
    sr_est_price_early = df_stock_rating_early3.groupby('sid', as_index=True)['s_est_price'].last()
    sr_est_price_last = df_stock_rating_p5d3.groupby('sid', as_index=True)['s_est_price'].last()
    sr_est_price_delta = sr_est_price_last - sr_est_price_early  # 目标价周下调
    
    # 预期eps周下调， 这张表每天更新，跑起来需要1~2min,所以用past 1month
    sql = ("""select S_INFO_WINDCODE, EST_DT, ROLLING_TYPE, EST_EPS from ASHARECONSENSUSROLLINGDATA"""
           + """ where ROLLING_TYPE='FY1' and EST_DT>='%s' and EST_DT<='%s'""" % (date_p1m, date))
    print('Reading sql AShareConsensusRollingData')
    df_con_eps = pd.read_sql(sql, conn)
    df_con_eps.columns = [ele.lower() for ele in df_con_eps.columns]
    df_con_eps.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_con_eps.est_dt = df_con_eps.est_dt.astype('i4')
    df_con_eps = df_con_eps[df_con_eps.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_con_eps.sort_values(['sid', 'est_dt'], inplace=True)
    # 处理数据
    df_con_eps_early = df_con_eps[df_con_eps.est_dt <= date_p5d]
    df_con_eps_early = df_con_eps_early[~np.isnan(df_con_eps_early.est_eps)]
    df_con_eps_p5d = df_con_eps[df_con_eps.est_dt > date_p5d]
    df_con_eps_p5d = df_con_eps_p5d[~np.isnan(df_con_eps_p5d.est_eps)]
    sr_eps_early = df_con_eps_early.groupby('sid', as_index=True)['est_eps'].last()
    sr_eps_last = df_con_eps_p5d.groupby('sid', as_index=True)['est_eps'].last()
    sr_eps_delta = sr_eps_last - sr_eps_early  # 预期eps周下调
    
    # 输出结果
    return sr_rating_delta, sr_est_price_delta, sr_eps_delta

In [7]:
print(pd.datetime.now())
a, b, c = get_analyst_expect_info(date, conn)
print(pd.datetime.now())

2019-11-29 00:11:51.383567
Reading sql AShareStockRatingConsus
Reading sql AShareConsensusRollingData
2019-11-29 00:13:52.113275


In [8]:
## 减持（大股东减持、高管减持）
def get_holder_trade_info(date, conn):
    date_p3m = get_prev_n_trade_date(date, 60)  #先获取近3个月的数据
    
    # 先获取总股本，单位为万股
    sql = ("""select S_INFO_WINDCODE, TRADE_DT, TOT_SHR_TODAY from ASHAREEODDERIVATIVEINDICATOR"""
           + """ where TRADE_DT='%s'""" % date)
    print('Reading sql AShareEODDerivativeIndicator')
    df_tot_share = pd.read_sql(sql, conn)
    df_tot_share.columns = [ele.lower() for ele in df_tot_share.columns]
    df_tot_share.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_tot_share.trade_dt = df_tot_share.trade_dt.astype('i4')
    df_tot_share = df_tot_share[df_tot_share.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_tot_share.sort_values(['sid', 'trade_dt'], inplace=True)
    sr_tot_share = 10000.0 * df_tot_share.groupby('sid')['tot_shr_today'].last()
    
    # 大股东减持，里面的单位为股
    sql = ("""select S_INFO_WINDCODE, ANN_DT, HOLDER_TYPE, TRANSACT_TYPE, TRANSACT_QUANTITY, NEW_HOLD_TOT from ASHAREMJRHOLDERTRADE"""
           + """ where TRANSACT_TYPE='减持' and ANN_DT>='%s' and ANN_DT<='%s'""" % (date_p3m, date)) # 因为有起始日期和截至日期，所以我们还是以ann_dt为准
    print('Reading sql AShareMjrHolderTrade')
    df_holder_trade = pd.read_sql(sql, conn)
    df_holder_trade.columns = [ele.lower() for ele in df_holder_trade.columns]
    df_holder_trade.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_holder_trade.ann_dt = df_holder_trade.ann_dt.astype('i4')
    df_holder_trade = df_holder_trade[df_holder_trade.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_holder_trade.sort_values(['sid', 'ann_dt'], inplace=True)
    # 大股东减持
    df_holder_trade = df_holder_trade.merge(sr_tot_share, on='sid', how='left')
    df_holder_trade['new_hold_tot'] = df_holder_trade['new_hold_tot'].fillna(0.0)
    df_holder_trade['trade_ratio'] = 1.0 * df_holder_trade.transact_quantity / df_holder_trade.tot_shr_today
    df_holder_trade2 = df_holder_trade[df_holder_trade.new_hold_tot + df_holder_trade.transact_quantity >= 0.05*df_holder_trade.tot_shr_today]
    sr_holder_sell = df_holder_trade2.groupby('sid')['trade_ratio'].sum()
    
    # 高管减持
    df_holder_trade2 = df_holder_trade[df_holder_trade.holder_type=='3']
    sr_manager_sell = df_holder_trade2.groupby('sid')['trade_ratio'].sum()
    
    # 返回结果
    return sr_holder_sell, sr_manager_sell  #注意，如果得到总股本为np.nan，即使有减持，也会返回减持结果为0，我们暂且忽略

In [9]:
print(pd.datetime.now())
a, b = get_holder_trade_info(date, conn)
print(pd.datetime.now())

2019-11-29 00:13:52.187572
Reading sql AShareEODDerivativeIndicator
Reading sql AShareMjrHolderTrade
2019-11-29 00:13:53.133835


In [10]:
## 重组失败，限售股解禁，债券违约、问询函
def get_risk_event_info(date, conn):
    date_p1m = get_prev_n_trade_date(date, 20)
    date_p6m = get_prev_n_trade_date(date, 120)
    date_p2y = get_prev_n_trade_date(date, 480)
    date_f1m = get_next_n_trade_date(date, 20+5)
    
    # 重组失败
    sql="""select EVENT_ID, S_INFO_WINDCODE from MERGERPARTICIPANT"""  # 获取关联并购的参与方
    print('Reading sql MergerParticipant')
    df_merger_participant = pd.read_sql(sql,conn)
    df_merger_participant.columns = [ele.lower() for ele in df_merger_participant.columns]
    df_merger_participant.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_merger_participant = df_merger_participant[df_merger_participant.sid.apply(lambda x:x is not None and ((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_merger_participant = df_merger_participant.groupby(['event_id', 'sid'], as_index=True).last().reset_index()
    # 获取关联并购
    sql = ("""select EVENT_ID, ANN_DATE, UPDATE_DATE, IS_MAJORASSETRESTRUCTURE, PROGRESS_CODE, TRADE_VALUE, CRNCY_CODE from MERGEREVENT"""
           + """ where IS_MAJORASSETRESTRUCTURE=1 and UPDATE_DATE>='%s' and UPDATE_DATE<='%s'""" % (date_p6m, date))
    print('Reading sql MergerEvent')
    df_merger_event = pd.read_sql(sql,conn)
    df_merger_event.columns = [ele.lower() for ele in df_merger_event.columns]
    df_merger_event.update_date = df_merger_event.update_date.astype('i4')
    df_merger_event = df_merger_event.merge(df_merger_participant, on='event_id', how='inner')  #合并，取inner
    df_merger_event = df_merger_event[df_merger_event.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_merger_event.sort_values(['sid', 'update_date'], inplace=True)
    # 处理数据
    df_merger_event = df_merger_event[df_merger_event.progress_code.apply(lambda x: x in  [324005000, 324005001, 324005002, 324005003, 324005004, 324005005])]
    sr_merger_failure = 1.0 * (df_merger_event.groupby('sid').apply(len) > 0) # 重组失败
    
    # 限售股解禁, 2010年以来存在4209/3593632约0.117%的情况ANN_DT为Null，我们暂且忽略
    sql=("""select S_INFO_WINDCODE, ANN_DT, S_INFO_LISTDATE, S_SHARE_LST, S_SHARE_RATIO from ASHARECOMPRESTRICTED"""
         + """ where ANN_DT<='%s' and S_INFO_LISTDATE>'%s' and S_INFO_LISTDATE<='%s'""" % (date, date, date_f1m))
    print('Reading sql AShareCompRestricted')
    df_restricted_list = pd.read_sql(sql,conn)
    df_restricted_list.columns = [ele.lower() for ele in df_restricted_list.columns]
    df_restricted_list.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_restricted_list.ann_dt = df_restricted_list.ann_dt.astype('i4')
    df_restricted_list = df_restricted_list[df_restricted_list.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_restricted_list.sort_values(['sid', 'ann_dt'], inplace=True)
    # 计算
    sr_restricted_ratio = 0.01 * df_restricted_list.groupby('sid')['s_share_ratio'].sum() # 限售股解禁
    
    # 债券违约
    sql=("""select S_INFO_WINDCODE, S_EVENT_ANNCEDATE, S_EVENT_CONTENT from ASHAREMAJOREVENT""" 
         + """ where S_EVENT_ANNCEDATE>='%s' and S_EVENT_ANNCEDATE<='%s'""" % (date_p2y, date))
    print('Reading sql AShareMajorEvent')
    df_major_event = pd.read_sql(sql, conn)
    df_major_event.columns = [ele.lower() for ele in df_major_event.columns]
    df_major_event.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_major_event.s_event_anncedate = df_major_event.s_event_anncedate.astype('i4')
    df_major_event = df_major_event[df_major_event.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_major_event.sort_values(['sid', 's_event_anncedate'], inplace=True)
    # 处理
    bond_default_list = ['担保违约','担保人代偿','兑付风险警示','未按时兑付本金','未按时兑付利息','未按时兑付本息',
                         '未按时兑付回售款', '未按时兑付回售款和利息', '提前到期未兑付','技术性违约']
    def func_bond_default(x):
        if x is None:
            return False
        for ele in bond_default_list:
            if ele in x:
                return True
        return ('债券' in x and '违约' in x)
    df_major_event2 = df_major_event[df_major_event.s_event_content.apply(lambda x: func_bond_default(x))]
    sr_bond_default = 1.0 * (df_major_event2.groupby('sid').apply(len) > 0)
    
    #问询函
    df_major_event3 = df_major_event[df_major_event.s_event_anncedate >= date_p1m]  # 最近一个月
    df_major_event3 = df_major_event3[df_major_event3.s_event_content.apply(lambda x: x is not None and '问询函' in x)]
    sr_enquiry_letter = 1.0 * (df_major_event3.groupby('sid').apply(len) > 0)
    
    return sr_merger_failure, sr_restricted_ratio, sr_bond_default, sr_enquiry_letter

In [11]:
print(pd.datetime.now())
a, b, c, d = get_risk_event_info(date, conn)
print(pd.datetime.now())

2019-11-29 00:13:53.285003
Reading sql MergerParticipant
Reading sql MergerEvent
Reading sql AShareCompRestricted
Reading sql AShareMajorEvent
2019-11-29 00:14:20.126725


In [12]:
## 财务报告（业绩实亏、业绩预亏、业绩预告大幅度下滑、审计意见，政府补助）
def get_finance_report_info(date, conn):
    date_p1y = get_prev_n_trade_date(date, 240)
    
    # 业绩实际亏损
    sql=(f"""select S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD, NET_PROFIT_INCL_MIN_INT_INC from ASHAREINCOME"""
         + f""" where STATEMENT_TYPE='408001000' and REPORT_PERIOD>='%s' and ANN_DT<='%s'""" % (date_p1y, date))
    print('Reading sql AShareIncome')
    df_income = pd.read_sql(sql,conn)
    df_income.columns = [ele.lower() for ele in df_income.columns]
    df_income.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_income.ann_dt = df_income.ann_dt.astype('i4')
    df_income.report_period = df_income.report_period.astype('i4')
    df_income = df_income[df_income.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))
                          & (~np.isnan(df_income.net_profit_incl_min_int_inc))]
    df_income.sort_values(['sid', 'report_period', 'ann_dt'], inplace=True)
    # 处理定期财务报告数据
    sr_income_period = df_income.groupby('sid')['report_period'].last()
    sr_profit = df_income.groupby('sid')['net_profit_incl_min_int_inc'].last()
    sr_actual_deficit = 1.0 * (sr_profit < 0.0)
    
    # 业绩预亏(对比了一下，上年同期归母净利润等三项单位都是万元)，需要观察预报是否领先于财报，领先财报的sid才算快报
    sql=(f"""select S_INFO_WINDCODE, S_PROFITNOTICE_DATE, S_PROFITNOTICE_PERIOD, S_PROFITNOTICE_NETPROFITMIN, S_PROFITNOTICE_NETPROFITMAX, S_PROFITNOTICE_NET_PARENT_FIRM from ASHAREPROFITNOTICE"""
         + f""" where S_PROFITNOTICE_PERIOD>='%s' and S_PROFITNOTICE_DATE<='%s'""" % (date_p1y, date))
    print('Reading sql AShareProfitNotice')
    df_profit_notice = pd.read_sql(sql,conn)
    df_profit_notice.columns = [ele.lower() for ele in df_profit_notice.columns]
    df_profit_notice.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_profit_notice.s_profitnotice_date = df_profit_notice.s_profitnotice_date.astype('i4')
    df_profit_notice.s_profitnotice_period = df_profit_notice.s_profitnotice_period.astype('i4')
    df_profit_notice = df_profit_notice[df_profit_notice.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_profit_notice.sort_values(['sid', 's_profitnotice_period', 's_profitnotice_date'], inplace=True)
    # 处理
    df_profit_notice = df_profit_notice.groupby('sid', as_index=True).last()
    df_profit_notice = df_profit_notice.merge(sr_income_period, on='sid', how='left')
    df_profit_notice.report_period = df_profit_notice.report_period.replace(np.nan, 0).astype('i4')
    df_profit_notice = df_profit_notice[df_profit_notice.s_profitnotice_period > df_profit_notice.report_period] # 预告要在财务报告之后
    sr_notice_deficit = 1.0 * (df_profit_notice.s_profitnotice_netprofitmin + df_profit_notice.s_profitnotice_netprofitmax < 0.0)  # 业绩预亏
    
    # 业绩大幅下滑
    def func_collapse(x):
        profit_notice_mean = 0.5 * (x.s_profitnotice_netprofitmin + x.s_profitnotice_netprofitmax)
        profit_pre = 1.0 * x.s_profitnotice_net_parent_firm
        if profit_pre < 0:
            return profit_notice_mean < 1.5 * profit_pre
        elif profit_pre >= 0:
            return profit_notice_mean < 0.5 * profit_pre
        else:   # np.nan
            return False
    sr_notice_collapse = 1.0 * pd.Series([func_collapse(ele) for _, ele in df_profit_notice.iterrows()], index=df_profit_notice.index)
    
    # 审计意见
    sql = ("""select S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD, S_STMNOTE_AUDIT_CATEGORY from ASHAREAUDITOPINION"""
           + f""" where REPORT_PERIOD>='%s' and ANN_DT<='%s'""" % (date_p1y, date))
    print('Reading sql AShareAuditOpinion')
    df_audit_opinion = pd.read_sql(sql,conn)
    df_audit_opinion.columns = [ele.lower() for ele in df_audit_opinion.columns]
    df_audit_opinion.rename({'s_info_windcode': 'sid'}, axis=1, inplace=True)
    df_audit_opinion.ann_dt = df_audit_opinion.ann_dt.astype('i4')
    df_audit_opinion.report_period = df_audit_opinion.report_period.astype('i4')
    df_audit_opinion = df_audit_opinion[df_audit_opinion.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_audit_opinion.sort_values(['sid', 'report_period', 'ann_dt'], inplace=True)
    # 处理数据
    df_audit_opinion = df_audit_opinion.groupby('sid', as_index=True).last()
    sr_audit_opinion = 1.0 * (df_audit_opinion.s_stmnote_audit_category != 405001000)    
    
    # 政府补助，所有项加在一起，表中单位为元，和公告也对了一下
    sql=(f"""select S_INFO_COMPCODE, ANN_DATE, REPORT_PERIOD, ITEM_NAME, AMOUNT_CURRENT_ISSUE from ASHAREGOVERNMENTGRANTS"""
         + f""" where REPORT_PERIOD>='%s' and ANN_DATE<='%s'""" % (date_p1y, date))
    print('Reading sql AShareGovernmentgrants')
    df_gov_grants= pd.read_sql(sql,conn)
    df_gov_grants.columns = [ele.lower() for ele in df_gov_grants.columns]
    df_gov_grants.ann_date = df_gov_grants.ann_date.astype('i4')
    df_gov_grants.report_period = df_gov_grants.report_period.astype('i4')
    df_gov_grants['sid'] = [code_map.get(ele, '0') for ele in df_gov_grants.s_info_compcode]
    df_gov_grants = df_gov_grants[df_gov_grants.sid.apply(lambda x:((x[::8] in ['0Z','3Z','6H']) & (len(x)==9)))]
    df_gov_grants.sort_values(['sid', 'report_period', 'ann_date'], inplace=True)
    # 处理数据
    sr_grants_amount = df_gov_grants.groupby(['sid', 'report_period'], as_index=True)['amount_current_issue'].sum().groupby('sid').last()
    sr_grants_ratio = sr_grants_amount / (np.abs(sr_profit) + 1.e4)  # 先这么定义吧
    
    return sr_actual_deficit, sr_notice_deficit, sr_notice_collapse, sr_audit_opinion, sr_grants_ratio

In [13]:
print(pd.datetime.now())
a, b, c, d, e = get_finance_report_info(date, conn)
print(pd.datetime.now())

2019-11-29 00:14:20.301207
Reading sql AShareIncome
Reading sql AShareProfitNotice
Reading sql AShareAuditOpinion
Reading sql AShareGovernmentgrants
2019-11-29 00:14:25.063042
